<nav style="margin-top: 20px; padding: 10px; background-color: #f0f0f0; border: 10px solid #ccc;">
    <div style="background-color:#2C41FF">
  <h1 style="margin: 0;text-align: center;">HyperParametersTuning</h1>
    </div>
  <ul style="list-style: none; padding: 0;">
    <li style="margin: 5px 0;"><a href="#preSec" style="text-decoration: none; color: #333;">Imports/Donwloads/Configuration </a></li>
    <li style="margin: 5px 0;"><a href="#sec1" style="text-decoration: none; color: #333;">HyperTuning QWK</a></li>
    <li style="margin: 5px 0;"><a href="#sec2" style="text-decoration: none; color: #333;">HyperTuning QWK + Thresholds</a></li>
  </ul>
</nav>


This notebook is used for reasereach of the hyperparameter of the models. 
It appears that those parameters are playing important roles in the models performances 
</br>
<a href="#back" style="text-decoration: none; color: #333;">Back to table of contents</a>

In [1]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib as plt
from tqdm import tqdm
from optuna.logging import set_verbosity, WARNING

##################### sklearn imports 
from scipy.optimize import minimize
from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score,confusion_matrix,make_scorer
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor

##################### Models
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

##################### optuna library import
import optuna
import shap
random_state = 42
n_splits = 5

c:\Users\Nadav\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
######## Download the data 
train=pd.read_csv("../data/processed/train_processed.csv")
test=pd.read_csv("../data/processed/test_processed.csv")
sample = pd.read_csv('../data/raw/sample_submission.csv')

In [3]:
y = train["sii"]
X = train.drop("sii",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')


def threshold_rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

qwk_scorer = make_scorer(quadratic_weighted_kappa, greater_is_better=True)

In [21]:
import optuna
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import cohen_kappa_score

# QWK Metric Function
def qwk_metric(y_true, y_pred, n_bins=5):
    """Compute Quadratic Weighted Kappa (QWK) between true and predicted values."""
    try:
        y_true = np.array(y_true).reshape(-1, 1)
        y_pred = np.array(y_pred).reshape(-1, 1)

        kbd = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
        y_true_binned = kbd.fit_transform(y_true).ravel()
        y_pred_binned = kbd.transform(y_pred).ravel()

        qwk = cohen_kappa_score(y_true_binned, y_pred_binned, weights='quadratic')
        return qwk
    except Exception as e:
        print("Error in QWK calculation:", e)
        return np.nan

# Custom QWK Scorer
qwk_scorer = make_scorer(qwk_metric, greater_is_better=True)

# Define Optuna Objectives
def objective_catboost(trial, X_train, y_train):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'random_strength': trial.suggest_uniform('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0, 1),
        'border_count': trial.suggest_int('border_count', 1, 255),
    }

    model = CatBoostRegressor(**params, verbose=0, early_stopping_rounds=50)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)
    return np.mean(score)

def objective_xgboost(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
        'min_child_weight': trial.suggest_int("min_child_weight", 1, 10),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10),
    }

    model = XGBRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)
    return np.mean(score)

def objective_lightgbm(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10),
    }

    model = LGBMRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)
    return np.mean(score)

def objective_randomforest(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
    }

    model = RandomForestRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)
    return np.mean(score)

def objective_gbr(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
    }

    model = GradientBoostingRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)
    return np.mean(score)

def objective_lr(trial, X_train, y_train):
    model = LinearRegression()
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)
    return np.mean(score)

# models
# for model in models


In [22]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective_xgboost(trial, X_train, y_train), n_trials=50)
print("Best parameters:", study.best_params)
print("Best score:", study.best_value)
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective_catboost(trial, X_train, y_train), n_trials=50)
print("Best parameters:", study.best_params)
print("Best score:", study.best_value)

[I 2024-12-14 22:30:19,345] A new study created in memory with name: no-name-7f1a0e31-34d7-40a6-8091-8b906fc398f9
C:\Users\Nadav\AppData\Local\Temp\ipykernel_13868\4120200244.py:54: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Nadav\AppData\Local\Temp\ipykernel_13868\4120200244.py:55: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1),
C:\Users\Nadav\AppData\Local\Temp\ipykernel_13868\4120200244.py:57: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0

Best parameters: {'n_estimators': 372, 'max_depth': 3, 'learning_rate': 0.11171122930966446, 'subsample': 0.7798809835378621, 'min_child_weight': 7, 'colsample_bytree': 0.779380569387379, 'reg_alpha': 0.565043616155518, 'reg_lambda': 3.911723931576487}
Best score: 0.3659203304437922


[I 2024-12-14 22:39:17,355] Trial 0 finished with value: 0.3148188694626411 and parameters: {'iterations': 312, 'depth': 5, 'learning_rate': 0.2774348576203086, 'l2_leaf_reg': 0.0626474436282604, 'random_strength': 7.66412359917145, 'bagging_temperature': 0.10662475160100504, 'border_count': 55}. Best is trial 0 with value: 0.3148188694626411.
[I 2024-12-14 22:40:22,263] Trial 1 finished with value: 0.3214530231275378 and parameters: {'iterations': 683, 'depth': 8, 'learning_rate': 0.2319382046642922, 'l2_leaf_reg': 0.11510861021914454, 'random_strength': 12.206788641071713, 'bagging_temperature': 0.9462153446741108, 'border_count': 88}. Best is trial 1 with value: 0.3214530231275378.
[I 2024-12-14 22:41:34,322] Trial 2 finished with value: 0.3147294487467487 and parameters: {'iterations': 927, 'depth': 7, 'learning_rate': 0.25668119896293673, 'l2_leaf_reg': 0.0017756475507477395, 'random_strength': 6.750596959554032, 'bagging_temperature': 0.5256822429684878, 'border_count': 137}. Bes

Best parameters: {'iterations': 994, 'depth': 6, 'learning_rate': 0.03209130827225604, 'l2_leaf_reg': 0.014027858863433254, 'random_strength': 3.1431517407535514, 'bagging_temperature': 0.10382819669361998, 'border_count': 145}
Best score: 0.36846444420759517


In [ ]:
# Generate a small dataset for testing
from sklearn.datasets import make_regression
import numpy as np

X_sample, y_sample = make_regression(n_samples=100, n_features=5, random_state=42)
y_sample_pred = np.random.randint(low=int(y_sample.min()), high=int(y_sample.max()), size=len(y_sample))

# Test the scorer directly
try:
    score = qwk_scorer(None, X_sample, y_sample)
    print("QWK Score:", score)
except Exception as e:
    print("Error in qwk_scorer:", e)
    

Error in qwk_scorer: 'NoneType' object has no attribute 'predict'


<section id="sec1"> </section>
<h1> HyperTuning QWK </h1>

In [ ]:
# CatBoost
def objective_catboost(trial, X_train, y_train):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'random_strength': trial.suggest_uniform('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0, 1),
        'border_count': trial.suggest_int('border_count', 1, 255),
    }

    model = CatBoostRegressor(**params, verbose=0, early_stopping_rounds=50)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

# XGBoost
def objective_xgboost(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
        'min_child_weight' : trial.suggest_int("min_child_weight", 1, 10),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10),
    }

    model = XGBRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)
    return np.mean(score)

# LightGBM
def objective_lightgbm(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10),
    }

    model = LGBMRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

# RandomForest
def objective_randomforest(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
    }

    model = RandomForestRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

# GradientBoosting
def objective_gbr(trial, X_train, y_train):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
    }

    model = GradientBoostingRegressor(**params)
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

# LinearRegression (no hyperparameters to tune)
def objective_lr(trial, X_train, y_train):
    model = LinearRegression()
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, scoring=qwk_scorer, cv=cv, n_jobs=-1)

    return np.mean(score)

In [ ]:
def HyperTuning_Model(model_name, X_train, y_train):
    if model_name == 'CatBoost':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_catboost(trial, X_train, y_train), n_trials=2)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")
        return best_params

    elif model_name == 'XGBoost':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_xgboost(trial, X_train, y_train), n_trials=2)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")
        return best_params

    elif model_name == 'LightGBM':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_lightgbm(trial, X_train, y_train), n_trials=2)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")
        return best_params

    elif model_name == 'RandomForest':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_randomforest(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")
        return best_params

    elif model_name == 'GradientBoosting':
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective_gbr(trial, X_train, y_train), n_trials=50)

        best_params = study.best_params
        print(f"Best parameters for {model_name}: {best_params}")
        return best_params


xgboost_best_params = HyperTuning_Model('XGBoost', X, y)
lgbm_best_params = HyperTuning_Model('LightGBM', X, y)
catboost_best_params = HyperTuning_Model('CatBoost', X, y)

<h1> HyperTuning QWK + thresholds </h1>

In [ ]:
# Set Optuna logging level to WARNING to reduce output
set_verbosity(WARNING)

# Define the objective function
def optuna_objective(trial, X, y, model_class, model_name):
    if "XGB" in model_name:
        # Suggest hyperparameters
        n_estimators = trial.suggest_int("n_estimators", 50, 300)
        max_depth = trial.suggest_int("max_depth", 2, 9)
        learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.1, log=True)
        subsample = trial.suggest_float("subsample", 0.5, 1.0)
        min_child_weight = trial.suggest_int("min_child_weight", 1, 10)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)
        reg_alpha = trial.suggest_float("reg_alpha", 0.5, 3)
        reg_lambda = trial.suggest_float("reg_lambda", 0.5, 10)

        # Initialize the model
        model = model_class(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_child_weight=min_child_weight,
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda
        )

    elif "CatBoost" in model_name:
        iterations = trial.suggest_int("iterations", 100, 1000)
        depth = trial.suggest_int("depth", 4, 10)
        learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.3)
        l2_leaf_reg = trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10)
        random_strength = trial.suggest_uniform("random_strength", 1, 20)
        bagging_temperature = trial.suggest_uniform("bagging_temperature", 0, 1)
        border_count = trial.suggest_int("border_count", 1, 255)

        # Initialize the model
        model = model_class(
            iterations=iterations,
            depth=depth,
            learning_rate=learning_rate,
            l2_leaf_reg=l2_leaf_reg,
            random_strength=random_strength,
            bagging_temperature=bagging_temperature,
            border_count=border_count,
        )

    elif "LGBM" in model_name:
        num_leaves = trial.suggest_int("num_leaves", 2, 256)
        learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.1, log=True)
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        min_child_samples = trial.suggest_int("min_child_samples", 5, 100)
        subsample = trial.suggest_float("subsample", 0.5, 1.0)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0)

        # Initialize the model
        model = model_class(
            num_leaves=num_leaves,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            min_child_samples=min_child_samples,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
        )

    # Split the data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit the model
    model.fit(X_train, y_train)

    # Predict non-rounded values
    oof_non_rounded = model.predict(X_val)

    # Optimize thresholds
    def optimize_thresholds(trial):
        thresholds = [
            trial.suggest_float(f"threshold_{i}", 0.5 * i, 0.5 * (i + 2)) for i in range(3)
        ]
        return evaluate_predictions(thresholds, y_val, oof_non_rounded)

    threshold_study = optuna.create_study(direction="minimize")
    threshold_study.optimize(optimize_thresholds, n_trials=30)  # Adjust as needed

    # Best thresholds and tuned Kappa score
    best_thresholds = [threshold_study.best_params[f"threshold_{i}"] for i in range(3)]
    tuned_kappa = -threshold_study.best_value  # Negative because of minimization

    return tuned_kappa


# Example usage
def run_hyperparameter_optimization(X, y, model_class):
    model_name = model_class.__name__
    study = optuna.create_study(direction="maximize")  # Maximize tuned Kappa score
    study.optimize(lambda trial: optuna_objective(trial, X, y, model_class, model_name), n_trials=50)
    print("Best hyperparameters:", study.best_params)
    print("Best Tuned Kappa score:", study.best_value)
    return study

In [15]:
study_result_LGBM = run_hyperparameter_optimization(X, y, LGBMRegressor)
study_result_XGB = run_hyperparameter_optimization(X, y, XGBRegressor)
study_result_CatBoost = run_hyperparameter_optimization(X, y, CatBoostRegressor)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002449 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20579
[LightGBM] [Info] Number of data points in the train set: 2188, number of used features: 125
[LightGBM] [Info] Start training from score 0.587751
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001792 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20579
[LightGBM] [Info] Number of data points in the train set: 2188, number of used features: 125
[LightGBM] [Info] Start training from score 0.587751
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.3)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_leaf_reg = trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  random_strength = trial.suggest_uniform("random_strength", 1

0:	learn: 0.7252811	total: 169ms	remaining: 29.8s
1:	learn: 0.6952777	total: 194ms	remaining: 17s
2:	learn: 0.6742782	total: 215ms	remaining: 12.4s
3:	learn: 0.6595638	total: 234ms	remaining: 10.1s
4:	learn: 0.6457417	total: 255ms	remaining: 8.76s
5:	learn: 0.6336807	total: 275ms	remaining: 7.83s
6:	learn: 0.6232535	total: 298ms	remaining: 7.23s
7:	learn: 0.6138916	total: 320ms	remaining: 6.76s
8:	learn: 0.6088308	total: 343ms	remaining: 6.41s
9:	learn: 0.6035825	total: 361ms	remaining: 6.03s
10:	learn: 0.5977630	total: 426ms	remaining: 6.43s
11:	learn: 0.5941889	total: 498ms	remaining: 6.85s
12:	learn: 0.5912900	total: 521ms	remaining: 6.57s
13:	learn: 0.5840943	total: 539ms	remaining: 6.27s
14:	learn: 0.5813074	total: 566ms	remaining: 6.11s
15:	learn: 0.5795752	total: 594ms	remaining: 5.98s
16:	learn: 0.5749208	total: 617ms	remaining: 5.8s
17:	learn: 0.5719406	total: 634ms	remaining: 5.6s
18:	learn: 0.5702953	total: 650ms	remaining: 5.41s
19:	learn: 0.5688075	total: 669ms	remaining: 

C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.3)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_leaf_reg = trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  random_strength = trial.suggest_uniform("random_strength", 1

0:	learn: 0.7599433	total: 6.17ms	remaining: 617ms
1:	learn: 0.7525158	total: 10.2ms	remaining: 505ms
2:	learn: 0.7404851	total: 13.7ms	remaining: 447ms
3:	learn: 0.7330536	total: 17ms	remaining: 412ms
4:	learn: 0.7271962	total: 20.5ms	remaining: 393ms
5:	learn: 0.7246494	total: 23.4ms	remaining: 370ms
6:	learn: 0.7183327	total: 26.4ms	remaining: 354ms
7:	learn: 0.7142144	total: 29.6ms	remaining: 344ms
8:	learn: 0.7089088	total: 31.9ms	remaining: 326ms
9:	learn: 0.7075802	total: 34.3ms	remaining: 312ms
10:	learn: 0.7048374	total: 36.5ms	remaining: 299ms
11:	learn: 0.7027081	total: 39ms	remaining: 289ms
12:	learn: 0.7008655	total: 41.3ms	remaining: 279ms
13:	learn: 0.6976475	total: 45ms	remaining: 280ms
14:	learn: 0.6959113	total: 47.8ms	remaining: 274ms
15:	learn: 0.6953731	total: 50.3ms	remaining: 267ms
16:	learn: 0.6939912	total: 52.6ms	remaining: 260ms
17:	learn: 0.6927971	total: 55ms	remaining: 254ms
18:	learn: 0.6911264	total: 57.4ms	remaining: 248ms
19:	learn: 0.6897282	total: 61

...

0:	learn: 0.7665612	total: 13.2ms	remaining: 11.4s
1:	learn: 0.7650154	total: 19ms	remaining: 8.25s
2:	learn: 0.7632240	total: 40ms	remaining: 11.6s
3:	learn: 0.7618657	total: 64.1ms	remaining: 13.9s
4:	learn: 0.7603886	total: 141ms	remaining: 24.4s
5:	learn: 0.7592571	total: 175ms	remaining: 25.3s
6:	learn: 0.7582534	total: 192ms	remaining: 23.7s
7:	learn: 0.7572295	total: 205ms	remaining: 22.1s
8:	learn: 0.7557771	total: 222ms	remaining: 21.3s
9:	learn: 0.7543223	total: 232ms	remaining: 20s
10:	learn: 0.7528358	total: 248ms	remaining: 19.4s
11:	learn: 0.7512473	total: 255ms	remaining: 18.3s
12:	learn: 0.7500699	total: 265ms	remaining: 17.5s
13:	learn: 0.7488384	total: 277ms	remaining: 16.9s
14:	learn: 0.7475889	total: 310ms	remaining: 17.7s
15:	learn: 0.7463444	total: 322ms	remaining: 17.2s
16:	learn: 0.7447271	total: 329ms	remaining: 16.5s
17:	learn: 0.7438057	total: 338ms	remaining: 16s
18:	learn: 0.7428121	total: 345ms	remaining: 15.5s
19:	learn: 0.7414639	total: 353ms	remaining: 

C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.3)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_leaf_reg = trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  random_strength = trial.suggest_uniform("random_strength", 1

0:	learn: 0.7670831	total: 9.74ms	remaining: 5.27s
1:	learn: 0.7657105	total: 15.2ms	remaining: 4.09s
2:	learn: 0.7647198	total: 20.5ms	remaining: 3.69s
3:	learn: 0.7628769	total: 25.8ms	remaining: 3.47s
4:	learn: 0.7611559	total: 30.7ms	remaining: 3.3s
5:	learn: 0.7594426	total: 37ms	remaining: 3.31s
6:	learn: 0.7581639	total: 46ms	remaining: 3.52s
7:	learn: 0.7566293	total: 52.2ms	remaining: 3.48s
8:	learn: 0.7549130	total: 60.6ms	remaining: 3.59s
9:	learn: 0.7538721	total: 132ms	remaining: 7.03s
10:	learn: 0.7525323	total: 226ms	remaining: 10.9s
11:	learn: 0.7514095	total: 239ms	remaining: 10.6s
12:	learn: 0.7503685	total: 284ms	remaining: 11.5s
13:	learn: 0.7488331	total: 298ms	remaining: 11.2s
14:	learn: 0.7475496	total: 308ms	remaining: 10.8s
15:	learn: 0.7474286	total: 315ms	remaining: 10.4s
16:	learn: 0.7465609	total: 322ms	remaining: 9.93s
17:	learn: 0.7461545	total: 328ms	remaining: 9.54s
18:	learn: 0.7453098	total: 335ms	remaining: 9.23s
19:	learn: 0.7443427	total: 344ms	rem

C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.3)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_leaf_reg = trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10)
C:\Users\Nadav\AppData\Local\Temp\ipykernel_25612\2458957604.py:30: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  random_strength = trial.suggest_uniform("random_strength", 1

0:	learn: 0.7678627	total: 7.16ms	remaining: 2.8s
1:	learn: 0.7660226	total: 11.5ms	remaining: 2.23s
2:	learn: 0.7623139	total: 15ms	remaining: 1.95s
3:	learn: 0.7606019	total: 21ms	remaining: 2.04s
4:	learn: 0.7587775	total: 39.7ms	remaining: 3.08s
5:	learn: 0.7573219	total: 55.2ms	remaining: 3.55s
6:	learn: 0.7570370	total: 74.2ms	remaining: 4.08s
7:	learn: 0.7564926	total: 113ms	remaining: 5.44s
8:	learn: 0.7541100	total: 139ms	remaining: 5.93s
9:	learn: 0.7520229	total: 154ms	remaining: 5.89s
10:	learn: 0.7508297	total: 160ms	remaining: 5.55s
11:	learn: 0.7485738	total: 167ms	remaining: 5.29s
12:	learn: 0.7468939	total: 178ms	remaining: 5.18s
13:	learn: 0.7450516	total: 189ms	remaining: 5.1s
14:	learn: 0.7441656	total: 196ms	remaining: 4.92s
15:	learn: 0.7421182	total: 203ms	remaining: 4.78s
16:	learn: 0.7406184	total: 213ms	remaining: 4.7s
17:	learn: 0.7390357	total: 223ms	remaining: 4.64s
18:	learn: 0.7373269	total: 233ms	remaining: 4.57s
19:	learn: 0.7357063	total: 243ms	remaini

In [ ]:
for stud in [study_result_LGBM,study_result_CatBoost,study_result_XGB]:
    display(stud.best_value)
    display(stud.best_params)

0.5159989218536416

{'num_leaves': 6,
 'learning_rate': 0.01289932165869906,
 'n_estimators': 338,
 'min_child_samples': 33,
 'subsample': 0.5000017866088126,
 'colsample_bytree': 0.6837596526579736}

0.5080496430818633

{'iterations': 484,
 'depth': 4,
 'learning_rate': 0.017996929503222906,
 'l2_leaf_reg': 0.0024072902514459277,
 'random_strength': 1.015334564941212,
 'bagging_temperature': 0.3919622777782532,
 'border_count': 50}

0.5162252663622526

{'n_estimators': 250,
 'max_depth': 2,
 'learning_rate': 0.028961469535241206,
 'subsample': 0.9695579026623281,
 'min_child_weight': 8,
 'colsample_bytree': 0.9754605932147322}

In [ ]:
#### Previous tuning results
#xgb_params = {'n_estimators': 52, 'max_depth': 3, 'learning_rate': 0.03793049558991992, 'subsample': 0.9266446256927953}
xgb_params = {'n_estimators': 271, 'max_depth': 2, 'learning_rate': 0.01689768605965407, 'subsample': 0.9955767677642268, 'min_child_weight': 5, 'colsample_bytree': 0.8995272248542203}
cat_params = {'iterations': 299, 'depth': 5, 'learning_rate': 0.03865350746087135, 'l2_leaf_reg': 0.030256773871165415, 'random_strength': 8.689200240903318, 'bagging_temperature': 0.12479378511663052, 'border_count': 28}
#lgbm_params ={'n_estimators': 151, 'max_depth': 16, 'learning_rate': 0.03605099786509202, 'subsample': 0.9277507651988509}
lgbm_params ={'num_leaves': 6, 'learning_rate': 0.011789952596053038, 'n_estimators': 278, 'min_child_samples': 100, 'subsample': 0.6205594476303221, 'colsample_bytree': 0.7956527746964115}

<h3>Logging results</h3>
<ul>My features without the 0.1 corr - 0.3869/0.432</ul>
<ul>My features with everything normal - 0.3703/0.439</ul>
<ul>Without my features - 0.3775/0.437</ul>
<ul>Using autoencoder with my features - 0.3678/0.421</ul>
<ul>Using autoencoder without dropping corr - 0.3715/0.419</ul>
<ul>Using autoencoder without categorical features - 0.3759/0.421</ul>
<ul>hyperparameters catboost - 0.3663/0.44</ul>
<ul>hyperparameters xgboost - 0.3809/0.456</ul>
<ul>No hyperparameters - 0.3703/0.439</ul>
<ul>xgboost+lgbm first batch - 0.3823/0.469</ul>